In [ ]:

# Define the payload with the 'body' property
$payloadObject = @{
    body = '{ "username": "testuser", "password": "FastTrack#1", "email": "email@example.com" }'
}
# Convert the payload object to JSON
$payloadJson = $payloadObject | ConvertTo-Json
# Convert the JSON to a UTF-8 byte array
$bytes = [System.Text.Encoding]::UTF8.GetBytes($payloadJson)
# Convert the byte array to a Base64-encoded string
$base64 = [Convert]::ToBase64String($bytes)
# Invoke the Lambda function with the Base64-encoded payload
aws lambda invoke --function-name HandTermCdkStack-SignUpFunctionCFE784C5-qDLsuBvd1ODa --payload $base64 response.json 

In [ ]:
aws logs describe-log-groups

In [ ]:
aws logs describe-log-streams --log-group-name /aws/lambda/HandTermCdkStack-SignUpFunctionCFE784C5-qDLsuBvd1ODa --order-by LastEventTime --descending --limit 2

In [ ]:
aws logs filter-log-events `
    --log-group-name /aws/lambda/HandTermCdkStack-SignUpFunctionCFE784C5-qDLsuBvd1ODa `
    --filter-pattern '"ERROR"'

In [ ]:
aws logs get-log-events `
  --log-group-name /aws/lambda/HandTermCdkStack-SignUpFunctionCFE784C5-qDLsuBvd1ODa `
  --log-stream-name '2024/06/29/[$LATEST]d32288a07db64f3f85e0a95de858b4c9' --limit 10


In [ ]:
aws apigateway get-rest-apis | Set-Variable apisYaml
$apisYaml

In [ ]:
$apisYaml | ConvertFrom-Yaml | Select-Object { $_.items } | Set-Variable apis
$apis | select-object $_.items

In [ ]:
aws apigateway get-resources --rest-api-id u4ceo7qshi

In [ ]:
aws apigateway get-method --rest-api-id u4ceo7qshi --resource-id p1xomh011j --http-method OPTIONS

In [ ]:
curl -X POST 'https://u4ceo7qshi.execute-api.us-east-1.amazonaws.com/prod/signup'  'Content-Type: application/json' -d '{"username": "example1@example.com", "password": "Pass_w0rd", "email": "example1@example.com"}'

In [ ]:
curl 'https://u4ceo7qshi.execute-api.us-east-1.amazonaws.com/prod/signUp' `
  -X 'OPTIONS' `
  -H 'accept: */*' `
  -H 'accept-language: en-US,en;q=0.9' `
  -H 'access-control-request-headers: access-control-allow-origin,content-type' `
  -H 'access-control-request-method: POST' `
  -H 'origin: http://localhost:5173' `
  -H 'priority: u=1, i' `
  -H 'referer: http://localhost:5173/' `
  -H 'sec-fetch-dest: empty' `
  -H 'sec-fetch-mode: cors' `
  -H 'sec-fetch-site: cross-site' `
  -H 'user-agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36 Edg/126.0.0.0'

In [ ]:
curl 'https://u4ceo7qshi.execute-api.us-east-1.amazonaws.com/prod/signUp' `
  -H 'sec-ch-ua: "Not/A)Brand";v="8", "Chromium";v="126", "Microsoft Edge";v="126"' `
  -H 'sec-ch-ua-mobile: ?0' `
  -H 'User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36 Edg/126.0.0.0' `
  -H 'Content-Type: application/json' `
  -H 'Access-Control-Allow-Origin: *' `
  -H 'Accept: application/json, text/plain, */*' `
  -H 'Referer: http://localhost:5173/' `
  -H 'sec-ch-ua-platform: "Windows"' `
  --data-raw '{"username":"mark@bizkite.co","password":"Pass_w0rd","email":"mark@bizkite.co"}'

In [ ]:
aws cognito-idp list-users --user-pool-id us-east-1_3uSNUrQbE --region us-east-1 --profile mark

In [ ]:
aws apigateway get-deployments --rest-api-id u4ceo7qshii

In [ ]:
aws cloudformation list-stack-resources --stack-name HandTermCdkStack --region us-east-1 --query "StackResourceSummaries[?ResourceType=='AWS::Lambda::Function' && contains(PhysicalResourceId, 'GetLog')].PhysicalResourceId" --output text | Set-Variable logGroupName
$logGroupName = '/aws/lambda/' + $logGroupName
$logGroupName

In [ ]:
aws logs describe-log-groups --log-group-name-prefix $logGroupName 

In [ ]:
$latestLogStreamName = aws logs describe-log-streams --log-group-name $logGroupName --order-by LastEventTime --descending --max-items 1 --query "logStreams[0].logStreamName" --output text
$latestLogStreamName

In [ ]:
aws logs get-log-events --log-group-name $logGroupName --log-stream-name $latestLogStreamName --limit 10

In [2]:
# Define the base S3 URI
$baseUri = "s3://handterm/"
$bucketName = "handterm"
$prefix = "user_data/"
$logType = @('char-time_';'command_')

# Function to move files with the specified date pattern
function Move-S3Files {
    param (
        [string]$BucketName,
        [string]$Prefix
    )

    # Continuation token for paginated results
    $continuationToken = $null

    do {
        # List all objects in the S3 bucket and prefix

        Write-Host "Getting all files in $($listObjectsParams.Bucket)"
        $response = aws s3api list-objects-v2 --bucket $BucketName --prefix $prefix --output json | ConvertFrom-Json

        Write-Host $response.Contents.length

        foreach ($object in $response.Contents) {
            # Write-Host "Found $($object.Key)"
            $objectKey = $object.Key
            # Check if the object key matches the pattern
            if ($objectKey -match "command_(\d{4})-(\d{2})-(\d{2})T(\d{2}:\d{2}:\d{2}\.\d{3}Z\.json)") {
                # Extract date components
                $year = $matches[1]
                $month = $matches[2]
                $day = $matches[3]

                # Construct the new S3 object key
                $newKey = $objectKey -replace "command_(\d{4})-(\d{2})-(\d{2})", "char-time/$year/$month/$day/"

                # Write-Host "Moving file $objectKey to $newKey"
                # Move the file to the new location
                aws s3 mv "$baseUri$objectKey" "$baseUri$newKey"
            }
        }

        # Check if there is more data to process
        $continuationToken = if ($response.IsTruncated) { $response.NextContinuationToken } else { $null }

    } while ($continuationToken -ne $null)
}

# Call the function to move files recursively
Move-S3Files -BucketName $bucketName -Prefix $prefix

Getting all files in 
1634
Completed 622 Bytes/622 Bytes (1.3 KiB/s) with 1 file(s) remaining
move: s3://handterm/user_data/e4485438-e0d1-7098-10fe-cd1dd7c7885d/logs/command_2024-06-26T22:42:08.836Z.json to s3://handterm/user_data/e4485438-e0d1-7098-10fe-cd1dd7c7885d/logs/char-time/2024/06/26/T22:42:08.836Z.json
Completed 623 Bytes/623 Bytes (1.2 KiB/s) with 1 file(s) remaining
move: s3://handterm/user_data/e4485438-e0d1-7098-10fe-cd1dd7c7885d/logs/command_2024-06-26T22:42:22.781Z.json to s3://handterm/user_data/e4485438-e0d1-7098-10fe-cd1dd7c7885d/logs/char-time/2024/06/26/T22:42:22.781Z.json
Completed 609 Bytes/609 Bytes (1.2 KiB/s) with 1 file(s) remaining
move: s3://handterm/user_data/e4485438-e0d1-7098-10fe-cd1dd7c7885d/logs/command_2024-06-26T22:42:36.723Z.json to s3://handterm/user_data/e4485438-e0d1-7098-10fe-cd1dd7c7885d/logs/char-time/2024/06/26/T22:42:36.723Z.json
Completed 620 Bytes/620 Bytes (1.4 KiB/s) with 1 file(s) remaining
move: s3://handterm/user_data/e4485438-e0d1-7